GRUPO:
HENNY ROCIO CARRILLO - hcarril42467@universidadean.edu.co
WILBER ALEXANDER RODRIGUEZ CASTRO - wrodrig96546@universidadean.edu.co

TEMA: Tarea de modelos de conteo [CountVectorizer] [TfidfVectorizer]


In [3]:
import re ## Exprexiones regulares
import nltk ## Procesamiento de lenguaje natural
from nltk.corpus import stopwords ## Palabras vacias
from nltk.stem import SnowballStemmer ## Stemming
from nltk.tokenize import word_tokenize ## Tokenizacion
from nltk.tokenize import RegexpTokenizer ## Tokenizacion
from sklearn.feature_extraction.text import CountVectorizer ## Vectorizador
import spacy ## Procesamiento de lenguaje natural
import pandas as pd


############## Descarga de recursos de nltk ################
nltk.download('punkt') ## Tokenizador
nltk.download('stopwords') ## Palabras vacias
nltk.download('snowball_data') ## Stemming
nltk.download('wordnet') ## Lematizacion

############## Es necesario descargar estos recursos para poder ejecutar el script ################


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\analistaderiesgo1\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\analistaderiesgo1\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package snowball_data to
[nltk_data]     C:\Users\analistaderiesgo1\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package snowball_data is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\analistaderiesgo1\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# URL del archivo crudo desde GitHub
url = 'https://raw.githubusercontent.com/Izainea/nlp_ean/main/Datos/Datos%20Crudos/reviews_booking.csv'

# Cargar el archivo CSV en un DataFrame
Baseinicial_DF = pd.read_csv(url)

# Mostrar el DataFrame
Baseinicial_DF.head()

C:\Users\analistaderiesgo1\AppData\Local\Temp\ipykernel_33588\1439072273.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  Baseinicial_DF = pd.read_csv(url)


Unnamed: 0       País Acomodación    Noches  Fecha hospedaje  \
0           0  Venezuela         NaN  2 noches    julio de 2023   
1           1   Colombia         NaN  4 noches  octubre de 2024   
2           2   Colombia         NaN   1 noche  octubre de 2024   
3           3   Colombia         NaN   1 noche   agosto de 2024   
4           4   Colombia         NaN  3 noches    marzo de 2024   

              Grupo viaje           Fecha reseña  \
0               En pareja    31 de julio de 2023   
1  Persona que viaja sola  23 de octubre de 2024   
2  Persona que viaja sola  15 de octubre de 2024   
3               En pareja   20 de agosto de 2024   
4               En pareja    27 de marzo de 2024   

                                    Titulo Calificación  \
0                              Excepcional           10   
1                               Fantástica           10   
2                              Excepcional           10   
3  Gran apartamento en una gran ubicación.           10   
4                              Excepcional           10   

                                     Cosas Positivas Cosas Negativas reseña  
0  Fue la Mejor Opción que pudimos haber tomado e...             NaN     Sí  
1                  Ubicación, comodidades y limpieza             NaN     Sí  
2  El lugar esta muy bien ubicado, el apartamento...             NaN     Sí  
3  Todo estuvo perfecto. La ubicación y la relaci...             NaN     Sí  
4  Es muy cómodo y acogedor, la atención es excel...             NaN     Sí

In [4]:
# Eliminar filas con valores NaN en la columna 'Calificación'
Baseinicial_DF = Baseinicial_DF.dropna(subset=['Calificación'])

# Verificar si la columna 'Calificación' ahora está limpia
print(Baseinicial_DF['Calificación'].isnull().sum())

# Crear una nueva columna para segmentar las calificaciones
def segmentar_calificacion(calificación):
    try:
        calificación = float(calificación)  # Convertir a número (float o int)
        if 0 <= calificación <= 7:
            return "Negativas"
        elif 8 <= calificación <= 10:
            return "Positivas"
        else:
            return "Desconocido"
    except ValueError:
        return "Desconocido"

# Aplicar la segmentación
Baseinicial_DF['Segmento_Calificación'] = Baseinicial_DF['Calificación'].apply(segmentar_calificacion)

# Mostrar el DataFrame con la nueva columna
print(Baseinicial_DF.head())

0
   Unnamed: 0       País Acomodación    Noches  Fecha hospedaje  \
0           0  Venezuela         NaN  2 noches    julio de 2023   
1           1   Colombia         NaN  4 noches  octubre de 2024   
2           2   Colombia         NaN   1 noche  octubre de 2024   
3           3   Colombia         NaN   1 noche   agosto de 2024   
4           4   Colombia         NaN  3 noches    marzo de 2024   

              Grupo viaje           Fecha reseña  \
0               En pareja    31 de julio de 2023   
1  Persona que viaja sola  23 de octubre de 2024   
2  Persona que viaja sola  15 de octubre de 2024   
3               En pareja   20 de agosto de 2024   
4               En pareja    27 de marzo de 2024   

                                    Titulo Calificación  \
0                              Excepcional           10   
1                               Fantástica           10   
2                              Excepcional           10   
3  Gran apartamento en una gran ubicación.    

In [5]:
Baseinicial_DF['Calificación']=Baseinicial_DF['Calificación'].str.replace(',','.').astype(float)

In [6]:
pip install langdetect



     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ------------------------------------- 981.5/981.5 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993251 sha256=f1a86b279528924ee30a6a8d21b26c84520aba6e1f4e8aa36a568e7baa76d7ef
  Stored in directory: c:\users\analistaderiesgo1\appdata\local\pip\cache\wheels\c1\67\88\e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [7]:
from langdetect import detect

# Función para detectar el idioma de un texto
def detectar_idioma(texto):
    try:
        # Detectar el idioma
        idioma = detect(texto)
        # Retornar 'Español' si el idioma es español, y 'Otro' si no lo es
        return 'Español' if idioma == 'es' else 'Otro'
    except:
        return 'Otro'  # En caso de error, retornamos 'Otro'

# Filtrar por calificaciones menores o iguales a 7
Baseinicial_DFN = Baseinicial_DF[Baseinicial_DF['Calificación'] <= 7]

# Seleccionar solo la columna 'Cosas Negativas'
Baseinicial_DFN = Baseinicial_DFN[['Cosas Negativas', 'Calificación']]  # Incluir calificación para mantener contexto

# Filtrar textos con longitud mayor a 60 y manejar valores nulos
Baseinicial_DFN = Baseinicial_DFN[Baseinicial_DFN['Cosas Negativas'].fillna('').apply(len) > 60]

# Filtrar solo los textos que están en español
Baseinicial_DFN['Idioma'] = Baseinicial_DFN['Cosas Negativas'].apply(detectar_idioma)
Baseinicial_DFN = Baseinicial_DFN[Baseinicial_DFN['Idioma'] == 'Español']

# Tomar una muestra aleatoria de 7500
Baseinicial_DFN = Baseinicial_DFN.sample(7500)

# Añadir la columna 'Segmento_Calificación'
Baseinicial_DFN['Segmento_Calificación'] = 'Negativa'

# Mostrar las primeras filas para verificar
print(Baseinicial_DFN.head())




                                          Cosas Negativas  Calificación  \
224659  la decoracion del hotel, carece de ella , el a...           5.0   
3886    Mucha humedad en las habitaciones, incómodas, ...           4.0   
219366  !El desayuno es repetitivo y se hace notorio e...           7.0   
297676  El último día hicimos el check our y dejamos l...           4.0   
186089  La atencion es pesima y te dicen un precio y l...           3.0   

         Idioma Segmento_Calificación  
224659  Español              Negativa  
3886    Español              Negativa  
219366  Español              Negativa  
297676  Español              Negativa  
186089  Español              Negativa  


In [8]:
Baseinicial_DFN

Cosas Negativas  Calificación  \
224659  la decoracion del hotel, carece de ella , el a...           5.0   
3886    Mucha humedad en las habitaciones, incómodas, ...           4.0   
219366  !El desayuno es repetitivo y se hace notorio e...           7.0   
297676  El último día hicimos el check our y dejamos l...           4.0   
186089  La atencion es pesima y te dicen un precio y l...           3.0   
...                                                   ...           ...   
346004  La atención de la señora Martha en la recepció...           1.0   
223198  Las sábanas de las camas estaban sucias pues t...           7.0   
49833   Solo aceptan efectivo, como turista esto hace ...           3.0   
131106  Por favor fumiguen. Maté 15 zancudos en mi hab...           7.0   
30564   En el baño faltan muchos utensilios necesarios...           5.0   

         Idioma Segmento_Calificación  
224659  Español              Negativa  
3886    Español              Negativa  
219366  Español              Negativa  
297676  Español              Negativa  
186089  Español              Negativa  
...         ...                   ...  
346004  Español              Negativa  
223198  Español              Negativa  
49833   Español              Negativa  
131106  Español              Negativa  
30564   Español              Negativa  

[7500 rows x 4 columns]

In [9]:
# Función para detectar el idioma de un texto
def detectar_idioma(texto):
    try:
        # Detectar el idioma
        idioma = detect(texto)
        # Retornar 'Español' si el idioma es español, y 'Otro' si no lo es
        return 'Español' if idioma == 'es' else 'Otro'
    except:
        return 'Otro'  # En caso de error, retornamos 'Otro'

# Filtrar por calificaciones entre 8 y 10
Baseinicial_DFP = Baseinicial_DF[(Baseinicial_DF['Calificación'] >= 8) & (Baseinicial_DF['Calificación'] <= 10)]

# Seleccionar solo la columna 'Cosas Positivas'
Baseinicial_DFP = Baseinicial_DFP[['Cosas Positivas', 'Calificación']]  # Incluir calificación para mantener contexto

# Filtrar textos con longitud mayor a 60 y manejar valores nulos
Baseinicial_DFP = Baseinicial_DFP[Baseinicial_DFP['Cosas Positivas'].fillna('').apply(len) > 60]

# Filtrar solo los textos que están en español
Baseinicial_DFP['Idioma'] = Baseinicial_DFP['Cosas Positivas'].apply(detectar_idioma)
Baseinicial_DFP = Baseinicial_DFP[Baseinicial_DFP['Idioma'] == 'Español']

# Tomar una muestra aleatoria de 7500
Baseinicial_DFP = Baseinicial_DFP.sample(7500)

# Añadir la columna 'Segmento_Calificación'
Baseinicial_DFP['Segmento_Calificación'] = 'Positiva'

# Mostrar las primeras filas para verificar
print(Baseinicial_DFP.head())




                                          Cosas Positivas  Calificación  \
263869  Las instalaciones son súper lindas y modernas....          10.0   
343387  Excelentes instalaciones, muy comódas las habi...          10.0   
414311  Hotel muy bien situado. Instalaciones muy viej...           8.0   
97594   muy amable el personal, desayuno rico, pieza m...          10.0   
309725  La ubicacion es buena en todo el centro de la ...           8.0   

         Idioma Segmento_Calificación  
263869  Español              Positiva  
343387  Español              Positiva  
414311  Español              Positiva  
97594   Español              Positiva  
309725  Español              Positiva  


In [10]:
Baseinicial_DFP

Cosas Positivas  Calificación  \
263869  Las instalaciones son súper lindas y modernas....          10.0   
343387  Excelentes instalaciones, muy comódas las habi...          10.0   
414311  Hotel muy bien situado. Instalaciones muy viej...           8.0   
97594   muy amable el personal, desayuno rico, pieza m...          10.0   
309725  La ubicacion es buena en todo el centro de la ...           8.0   
...                                                   ...           ...   
372303  Las instalaciones son tal cual la de las imáge...           9.0   
407229  la cama deliciosa ! sin ningún tipo de ruido q...          10.0   
152023  Excelente ubicación, muy cerca de centros come...           9.0   
29461   El personal muy amable y el lugar muy bonito, ...          10.0   
622     Me gustó la atención, las camas, la habitación...          10.0   

         Idioma Segmento_Calificación  
263869  Español              Positiva  
343387  Español              Positiva  
414311  Español              Positiva  
97594   Español              Positiva  
309725  Español              Positiva  
...         ...                   ...  
372303  Español              Positiva  
407229  Español              Positiva  
152023  Español              Positiva  
29461   Español              Positiva  
622     Español              Positiva  

[7500 rows x 4 columns]

In [11]:
# Filtrar "Cosas Negativas" y añadir la columna 'Segmento_Calificación' como 'Negativa'
Baseinicial_DFN = Baseinicial_DF[Baseinicial_DF['Calificación'] <= 7]
Baseinicial_DFN = Baseinicial_DFN[['Cosas Negativas', 'Calificación']]  # Incluir Calificación
Baseinicial_DFN = Baseinicial_DFN[Baseinicial_DFN['Cosas Negativas'].fillna('').apply(len) > 60]
Baseinicial_DFN = Baseinicial_DFN.sample(7500)
Baseinicial_DFN['Segmento_Calificación'] = 'Negativa'

# Filtrar "Cosas Positivas" y añadir la columna 'Segmento_Calificación' como 'Positiva'
Baseinicial_DFP = Baseinicial_DF[(Baseinicial_DF['Calificación'] >= 8) & (Baseinicial_DF['Calificación'] <= 10)]
Baseinicial_DFP = Baseinicial_DFP[['Cosas Positivas', 'Calificación']]  # Incluir Calificación
Baseinicial_DFP = Baseinicial_DFP[Baseinicial_DFP['Cosas Positivas'].fillna('').apply(len) > 60]
Baseinicial_DFP = Baseinicial_DFP.sample(7500)
Baseinicial_DFP['Segmento_Calificación'] = 'Positiva'

# Crear una nueva columna 'Observaciones' para unificar las dos columnas de texto
Baseinicial_DFN['Observaciones'] = Baseinicial_DFN['Cosas Negativas']
Baseinicial_DFP['Observaciones'] = Baseinicial_DFP['Cosas Positivas']

# Ahora eliminamos las columnas originales ('Cosas Negativas' y 'Cosas Positivas')
Baseinicial_DFN = Baseinicial_DFN.drop(columns=['Cosas Negativas'])
Baseinicial_DFP = Baseinicial_DFP.drop(columns=['Cosas Positivas'])

# Unir ambos DataFrames (Cosas Negativas y Cosas Positivas) en un único DataFrame
Baseinicial_Consolidado = pd.concat([Baseinicial_DFN[['Observaciones', 'Calificación', 'Segmento_Calificación']],
                                     Baseinicial_DFP[['Observaciones', 'Calificación', 'Segmento_Calificación']]],
                                    ignore_index=True)

# Mostrar el resultado consolidado
print(Baseinicial_Consolidado.head())







                                       Observaciones  Calificación  \
0  The hotel was very old and the overall cleanli...           5.0   
1  ME CANCELARON LA RESERVA, POR LA HORA DE LLEGA...           1.0   
2  Absolut langweiliges Frühstücksbuffet, Persona...           5.0   
3  desde que llegue a bogota le informe a la sra ...           1.0   
4  Just about everything. No fans or air conditio...           4.0   

  Segmento_Calificación  
0              Negativa  
1              Negativa  
2              Negativa  
3              Negativa  
4              Negativa  


In [12]:
Baseinicial_Consolidado

Observaciones  Calificación  \
0      The hotel was very old and the overall cleanli...           5.0   
1      ME CANCELARON LA RESERVA, POR LA HORA DE LLEGA...           1.0   
2      Absolut langweiliges Frühstücksbuffet, Persona...           5.0   
3      desde que llegue a bogota le informe a la sra ...           1.0   
4      Just about everything. No fans or air conditio...           4.0   
...                                                  ...           ...   
14995  Very dark room after closing the curtains so p...           8.0   
14996  Gracias Carolina es una exlente anfitriona. Mu...          10.0   
14997  Cuando tienes conexión de vuelos es el hotel i...           9.0   
14998  Posizione comoda, camera pulita, spaziosa e be...          10.0   
14999  Muito bem localizado, bem no coração do bairro...          10.0   

      Segmento_Calificación  
0                  Negativa  
1                  Negativa  
2                  Negativa  
3                  Negativa  
4                  Negativa  
...                     ...  
14995              Positiva  
14996              Positiva  
14997              Positiva  
14998              Positiva  
14999              Positiva  

[15000 rows x 3 columns]

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer  # Si estás utilizando TfidfVectorizer

# Crear el objeto cleaner (vectorizador) 
cleaner = TfidfVectorizer(stop_words='english')  # 'english' elimina las stopwords comunes en inglés

# Asegúrate de que la columna 'Observaciones' no contenga valores nulos
Baseinicial_Consolidado['Observaciones'] = Baseinicial_Consolidado['Observaciones'].fillna('')

# Aplicar fit_transform a la columna 'Observaciones'
text_limpio = cleaner.fit_transform(Baseinicial_Consolidado['Observaciones'])

# Si necesitas el resultado como un array o DataFrame:
text_limpio_array = text_limpio.toarray()  # Convierte la matriz dispersa a un array denso
import pandas as pd
text_limpio_df = pd.DataFrame(text_limpio_array, columns=cleaner.get_feature_names_out())  # Convertir a DataFrame

# Ver las primeras filas del DataFrame resultante
print(text_limpio_df.head())


    00  000  000cop  000mil  00am  00h  00hh  00pm   01  0130h  ...  테이블이  \
0  0.0  0.0     0.0     0.0   0.0  0.0   0.0   0.0  0.0    0.0  ...   0.0   
1  0.0  0.0     0.0     0.0   0.0  0.0   0.0   0.0  0.0    0.0  ...   0.0   
2  0.0  0.0     0.0     0.0   0.0  0.0   0.0   0.0  0.0    0.0  ...   0.0   
3  0.0  0.0     0.0     0.0   0.0  0.0   0.0   0.0  0.0    0.0  ...   0.0   
4  0.0  0.0     0.0     0.0   0.0  0.0   0.0   0.0  0.0    0.0  ...   0.0   

   편안하고  플라스틱  피크닉  한식당이랑  할로윈때여서그런가  황금박물관  후원도   훨씬  \
0   0.0   0.0  0.0    0.0        0.0    0.0  0.0  0.0   
1   0.0   0.0  0.0    0.0        0.0    0.0  0.0  0.0   
2   0.0   0.0  0.0    0.0        0.0    0.0  0.0  0.0   
3   0.0   0.0  0.0    0.0        0.0    0.0  0.0  0.0   
4   0.0   0.0  0.0    0.0        0.0    0.0  0.0  0.0   

   ２４時間フリーの空港シャトルが１５分おきに使えること  
0                         0.0  
1                         0.0  
2                         0.0  
3                         0.0  
4                         0.0  

[